In [1]:
import pandas as pd
import numpy as np
#importing warnings to avoid unnecesaary warnings due to version change
import warnings
warnings.filterwarnings('ignore')
# importing labelencoder
from sklearn.preprocessing import OneHotEncoder
#importing for visualization process
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
test=pd.read_csv('test_data-1664552867678.csv')

In [3]:
test

,date,farm_id,ingredient_type,id
0,2017-01-01 00:00:00,fid_110884,ing_w,0
1,2017-01-01 00:00:00,fid_90053,ing_w,1
2,2017-01-01 00:00:00,fid_17537,ing_w,2
3,2017-01-01 00:00:00,fid_110392,ing_w,3
4,2017-01-01 00:00:00,fid_62402,ing_w,4
...,...,...,...,...
20848795,2017-12-25 07:00:00,fid_58769,ing_x,41696845
20848796,2017-12-25 07:00:00,fid_58769,ing_z,41696846
20848797,2017-12-25 07:00:00,fid_117010,ing_w,41696847
20848798,2017-12-25 07:00:00,fid_117010,ing_x,41696848


In [4]:
farm=pd.read_csv('farm_data-1646897931981.csv')

In [5]:
# merging the train and farm data with farm_id
test_farm = pd.merge(test,farm,on='farm_id')

In [6]:
#checking the null values of both farm and train
test_farm.isnull().sum()

date                                 0
farm_id                              0
ingredient_type                      0
id                                   0
operations_commencing_year    12526800
num_processing_plants         17520000
farm_area                            0
farming_company                      0
deidentified_location                0
dtype: int64

In [7]:
# dropping the null values
test_farm.drop('operations_commencing_year',axis=1,inplace=True)
test_farm.drop('num_processing_plants',axis=1,inplace=True)

In [8]:
#reading weather.csv file
weather = pd.read_csv('test_weather-1646897984996-1664552604982.csv')

In [9]:
weather.isnull().sum()

timestamp                    0
deidentified_location        0
temp_obs                    43
cloudiness               69065
wind_direction            6134
dew_temp                   184
pressure_sea_level       10883
precipitation            47788
wind_speed                 240
dtype: int64

In [10]:
#dropping the cloudiness and precipitation
weather.drop('cloudiness',axis=1,inplace=True)
weather.drop('precipitation',axis=1,inplace=True)

In [11]:
#filling the missing values with fillna method
weather['pressure_sea_level'].fillna('1015.2',inplace=True)

In [12]:
#imputing the missing values with 0 by fillna method
weather['wind_direction'].fillna(0,inplace=True)

In [13]:
#imputing the missing values with ffill method
weather['wind_speed'].fillna(method='ffill',inplace=True)

In [14]:
#imputing missing values by ffill
weather['temp_obs'].fillna(method='ffill',inplace=True)

In [15]:
#filling 50% of missing values with 10 and another 50% with 15
weather['dew_temp'].fillna('10',limit=92,inplace=True)
weather['dew_temp'].fillna('15',limit=92,inplace=True)

In [16]:
#checking the null values for verification
weather.isnull().sum()

timestamp                0
deidentified_location    0
temp_obs                 0
wind_direction           0
dew_temp                 0
pressure_sea_level       0
wind_speed               0
dtype: int64

In [17]:
#just to ensure that all the train dataset has a common name for timestamp as date, lets change the name of timestamp as date.
weather.rename(columns={'timestamp':'date'},inplace=True)

In [18]:
#merging weather&test_farm data with deidentified_location and date
weather_test_farm = pd.merge(test_farm,weather, on=['deidentified_location','date'])

In [19]:
#convering values in to int
dew_temp = weather_test_farm['dew_temp'].values
dew_temp1 = dew_temp.astype(int)
temp_obs = weather_test_farm['temp_obs'].values
temp_obs1 = temp_obs.astype(int)

In [20]:
#formulae for relative humidity
Relative_humidity = 100 * (np.exp((17.625 * dew_temp1)/(243.04 + dew_temp1))/np.exp((17.625 * temp_obs1)/(243.04 + temp_obs1)))

In [21]:
# Create a new pandas Series with the same index as the DataFrame
Relative_humidity = pd.Series(Relative_humidity, index=weather_test_farm.index)

# Assign the Series to a new column in the DataFrame
weather_test_farm['Relative_humidity'] = Relative_humidity
weather_test_farm

,date,farm_id,ingredient_type,id,farm_area,farming_company,deidentified_location,temp_obs,wind_direction,dew_temp,pressure_sea_level,wind_speed,Relative_humidity
0,2017-01-01 00:00:00,fid_110884,ing_w,0,690.455096,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,3.6,67.769404
1,2017-01-01 00:00:00,fid_90053,ing_w,1,252.696160,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,3.6,67.769404
2,2017-01-01 00:00:00,fid_17537,ing_w,2,499.446528,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,3.6,67.769404
3,2017-01-01 00:00:00,fid_110392,ing_w,3,2200.407555,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,3.6,67.769404
4,2017-01-01 00:00:00,fid_62402,ing_w,4,10833.140121,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,3.6,67.769404
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21154792,2017-12-31 23:00:00,fid_58769,ing_x,26001715,8659.117018,Obery Farms,location 868,-21.1,270.0,-26.8,1029.8,4.6,64.118800
21154793,2017-12-31 23:00:00,fid_58769,ing_z,26001716,8659.117018,Obery Farms,location 868,-21.1,270.0,-26.8,1029.8,4.6,64.118800
21154794,2017-12-31 23:00:00,fid_117010,ing_w,26001717,11857.395696,Obery Farms,location 868,-21.1,270.0,-26.8,1029.8,4.6,64.118800
21154795,2017-12-31 23:00:00,fid_117010,ing_x,26001718,11857.395696,Obery Farms,location 868,-21.1,270.0,-26.8,1029.8,4.6,64.118800


In [22]:
one_hot= OneHotEncoder(sparse=False,handle_unknown='ignore')

In [23]:
encoder=one_hot.fit_transform(weather_test_farm[['farming_company','deidentified_location']])
encoder=pd.DataFrame(encoder)
encoder.columns=one_hot.get_feature_names_out(['farming_company','deidentified_location'])

In [24]:
#concatting data and the encoder values
weather_test_farm=pd.concat([weather_test_farm,encoder],axis=1)

In [25]:
#dropping the columns after OneHotEncoding
weather_test_farm.drop(['farming_company','deidentified_location'],axis=1,inplace=True)

In [26]:
#converting to the dtypes according to the requirements
weather_test_farm['farm_id']=weather_test_farm['farm_id'].astype('category')
weather_test_farm['ingredient_type']=weather_test_farm['ingredient_type'].astype('category')
weather_test_farm['dew_temp']=weather_test_farm['dew_temp'].astype('float')
weather_test_farm['pressure_sea_level']=weather_test_farm['pressure_sea_level'].astype('float')

In [27]:
#downcasting the final data by converting according to the requirements
def downcast(weather_test_farm):
    cols = weather_test_farm.dtypes.index.tolist()
    types = weather_test_farm.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if weather_test_farm[cols[i]].min() > np.iinfo(np.int8).min and weather_test_farm[cols[i]].max() < np.iinfo(np.int8).max:
                weather_test_farm[cols[i]] = weather_test_farm[cols[i]].astype(np.int8)
            elif weather_test_farm[cols[i]].min() > np.iinfo(np.int16).min and weather_test_farm[cols[i]].max() < np.iinfo(np.int16).max:
                weather_test_farm[cols[i]] = weather_test_farm[cols[i]].astype(np.int16)
            elif weather_test_farm[cols[i]].min() > np.iinfo(np.int32).min and weather_test_farm[cols[i]].max() < np.iinfo(np.int32).max:
                weather_test_farm[cols[i]] = weather_test_farm[cols[i]].astype(np.int32)
            else:
                weather_test_farm[cols[i]] = weather_test_farm[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if weather_test_farm[cols[i]].min() > np.finfo(np.float16).min and weather_test_farm[cols[i]].max() < np.finfo(np.float16).max:
                weather_test_farm[cols[i]] = weather_test_farm[cols[i]].astype(np.float16)
            elif weather_test_farm[cols[i]].min() > np.finfo(np.float32).min and weather_test_farm[cols[i]].max() < np.finfo(np.float32).max:
                weather_test_farm[cols[i]] = weather_test_farm[cols[i]].astype(np.float32)
            else:
                weather_test_farm[cols[i]] = weather_test_farm[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                weather_test_farm[cols[i]] = pd.to_datetime(weather_test_farm[cols[i]], format='%Y-%m-%d')
            elif cols[i]=="timestamp":
                weather_test_farm[cols[i]] = pd.to_datetime(weather_test_farm[cols[i]], format='%Y-%m-%d')
            else:
                weather_test_farm[cols[i]] = weather_test_farm[cols[i]].astype('category')
    return weather_test_farm  

weather_test_farm = downcast(weather_test_farm)

In [28]:
#Ing_z
ing_z_test=weather_test_farm[weather_test_farm["ingredient_type"]=="ing_z"]
ing_z_test

,date,farm_id,ingredient_type,id,farm_area,temp_obs,wind_direction,dew_temp,pressure_sea_level,wind_speed,...,deidentified_location_location 5489,deidentified_location_location 565,deidentified_location_location 5677,deidentified_location_location 5833,deidentified_location_location 6364,deidentified_location_location 7048,deidentified_location_location 7369,deidentified_location_location 8421,deidentified_location_location 868,deidentified_location_location 959
1960832,2017-01-01 00:00:00,fid_87942,ing_z,2260082,499.260712,6.699219,200.0,5.199219,1024.0,5.101562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1960836,2017-01-01 00:00:00,fid_66870,ing_z,2260086,5295.006348,6.699219,200.0,5.199219,1024.0,5.101562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1960840,2017-01-01 00:00:00,fid_66062,ing_z,2260090,2992.033936,6.699219,200.0,5.199219,1024.0,5.101562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1960842,2017-01-01 00:00:00,fid_75323,ing_z,2260092,9334.986328,6.699219,200.0,5.199219,1024.0,5.101562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1960844,2017-01-01 00:00:00,fid_75397,ing_z,2260094,12976.969727,6.699219,200.0,5.199219,1024.0,5.101562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21154785,2017-12-31 23:00:00,fid_74945,ing_z,26001708,7561.375000,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154787,2017-12-31 23:00:00,fid_36739,ing_z,26001710,14173.188477,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154790,2017-12-31 23:00:00,fid_112848,ing_z,26001713,6320.190918,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154793,2017-12-31 23:00:00,fid_58769,ing_z,26001716,8659.117188,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [29]:
#Ing_x
ing_x_test=weather_test_farm[weather_test_farm["ingredient_type"]=="ing_x"]
ing_x_test

,date,farm_id,ingredient_type,id,farm_area,temp_obs,wind_direction,dew_temp,pressure_sea_level,wind_speed,...,deidentified_location_location 5489,deidentified_location_location 565,deidentified_location_location 5677,deidentified_location_location 5833,deidentified_location_location 6364,deidentified_location_location 7048,deidentified_location_location 7369,deidentified_location_location 8421,deidentified_location_location 868,deidentified_location_location 959
8,2017-01-01 00:00:00,fid_33785,ing_x,8,11248.137695,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
11,2017-01-01 00:00:00,fid_32954,ing_x,11,2508.381104,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16,2017-01-01 00:00:00,fid_43573,ing_x,16,9232.700195,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
18,2017-01-01 00:00:00,fid_48575,ing_x,18,8012.883789,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
20,2017-01-01 00:00:00,fid_109906,ing_x,20,7799.856934,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21154781,2017-12-31 22:00:00,fid_117010,ing_x,26001704,11857.395508,-20.796875,270.0,-26.796875,1029.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154784,2017-12-31 23:00:00,fid_74945,ing_x,26001707,7561.375000,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154789,2017-12-31 23:00:00,fid_112848,ing_x,26001712,6320.190918,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154792,2017-12-31 23:00:00,fid_58769,ing_x,26001715,8659.117188,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
#Ing_w
ing_w_test=weather_test_farm[weather_test_farm["ingredient_type"]=="ing_w"]
ing_w_test

,date,farm_id,ingredient_type,id,farm_area,temp_obs,wind_direction,dew_temp,pressure_sea_level,wind_speed,...,deidentified_location_location 5489,deidentified_location_location 565,deidentified_location_location 5677,deidentified_location_location 5833,deidentified_location_location 6364,deidentified_location_location 7048,deidentified_location_location 7369,deidentified_location_location 8421,deidentified_location_location 868,deidentified_location_location 959
0,2017-01-01 00:00:00,fid_110884,ing_w,0,690.455078,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2017-01-01 00:00:00,fid_90053,ing_w,1,252.696167,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2017-01-01 00:00:00,fid_17537,ing_w,2,499.446533,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2017-01-01 00:00:00,fid_110392,ing_w,3,2200.407471,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2017-01-01 00:00:00,fid_62402,ing_w,4,10833.139648,17.796875,100.0,11.703125,1021.5,3.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21154783,2017-12-31 23:00:00,fid_74945,ing_w,26001706,7561.375000,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154786,2017-12-31 23:00:00,fid_36739,ing_w,26001709,14173.188477,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154788,2017-12-31 23:00:00,fid_112848,ing_w,26001711,6320.190918,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21154791,2017-12-31 23:00:00,fid_58769,ing_w,26001714,8659.117188,-21.093750,270.0,-26.796875,1030.0,4.601562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [31]:
#Ing_y
ing_y_test=weather_test_farm[weather_test_farm["ingredient_type"]=="ing_y"]
ing_y_test

,date,farm_id,ingredient_type,id,farm_area,temp_obs,wind_direction,dew_temp,pressure_sea_level,wind_speed,...,deidentified_location_location 5489,deidentified_location_location 565,deidentified_location_location 5677,deidentified_location_location 5833,deidentified_location_location 6364,deidentified_location_location 7048,deidentified_location_location 7369,deidentified_location_location 8421,deidentified_location_location 868,deidentified_location_location 959
1138826,2017-01-01 01:00:00,fid_59158,ing_y,37198374,5664.946289,9.000000,240.0,7.000000,1015.0,3.599609,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1138920,2017-01-01 02:00:00,fid_59158,ing_y,37198624,5664.946289,9.000000,240.0,8.000000,1015.0,3.099609,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1139014,2017-01-01 03:00:00,fid_59158,ing_y,37198874,5664.946289,8.000000,270.0,8.000000,1015.0,3.599609,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1139108,2017-01-01 04:00:00,fid_59158,ing_y,37199124,5664.946289,8.000000,240.0,8.000000,1015.0,4.101562,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1139202,2017-01-01 05:00:00,fid_59158,ing_y,37199374,5664.946289,8.000000,200.0,8.000000,1015.0,2.099609,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20602160,2017-12-30 18:00:00,fid_31738,ing_y,34667909,6132.805664,-6.101562,200.0,-8.898438,1014.0,2.099609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20602163,2017-12-30 18:00:00,fid_121246,ing_y,34667912,5785.905762,-6.101562,200.0,-8.898438,1014.0,2.099609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20602166,2017-12-30 18:00:00,fid_77830,ing_y,34667915,5392.183105,-6.101562,200.0,-8.898438,1014.0,2.099609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20602169,2017-12-30 18:00:00,fid_64331,ing_y,34667918,5320.926270,-6.101562,200.0,-8.898438,1014.0,2.099609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
ing_w_test.to_csv('ing_w_test.csv', index=False)

In [33]:
ing_x_test.to_csv('ing_x_test.csv', index=False)

In [34]:
ing_y_test.to_csv('ing_y_test.csv', index=False)

In [35]:
ing_z_test.to_csv('ing_z_test.csv', index=False)